In [8]:
import sys; sys.path.append('..')
import numpy as np, elastic_rods
from bending_validation import suppress_stdout
from linkage_vis import LinkageViewer
linkage_name = '20190423_102656_meshID_1cec245f-3565-47aa-a78a-d6343c6ed811'
l = elastic_rods.RodLinkage('../../examples/florin/{}.obj'.format(linkage_name), 8)
#driver=166
driver=l.centralJoint()

#mat = elastic_rods.RodMaterial('+', 2000, 0.3, [5, 5, 0.7, 0.7], stiffAxis=elastic_rods.StiffAxis.D1)
mat = elastic_rods.RodMaterial('rectangle', 20000, 0.3, [12, 8])
l.setMaterial(mat)

with suppress_stdout(): elastic_rods.restlen_solve(l)
jdo = l.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
fixedVars.append(jdo + 6) # constrain angle at the driving joint
with suppress_stdout(): elastic_rods.compute_equilibrium(l, fixedVars=fixedVars)

view = LinkageViewer(l, width=1024)
#view.setCameraParams(((2.656451698624758, 1.5087349593639046, 0.23168476156681694),
# (0.052583904614345374, 0.026224725204501566, 0.9982721055721517),
# (0.13680349418553736, -0.13680349418553736, -0.8425278443781967)))
view.show()

Renderer(camera=PerspectiveCamera(aspect=2.0, children=(DirectionalLight(color='white', intensity=0.6, positio…

In [5]:
l.saveVisualizationGeometry('deployed_{}.obj'.format(linkage_name))
#l.writeLinkageDebugData('')

In [14]:
from write_render_files import writeRenderFiles, writeActuators
writeRenderFiles(l, 'Pavilion12', 'pav12')
#writeActuators(l, np.where(torques > 1e-4)[0], directory='AsymPointyDataFlat', name='asymPointy_Fab')

In [10]:
from linkage_utils import writeRodSegments
writeRodSegments(l,'rod_segments_{}.txt'.format(linkage_name), zeroBasedIndexing=True)
np.savetxt('restlen_{}.txt'.format(linkage_name),l.getPerSegmentRestLength())
np.savetxt('normals_{}.txt'.format(linkage_name), np.array([j.normal for j in l.joints()]))
l.writeLinkageDebugData('deployed_{}.msh'.format(linkage_name))

In [41]:
# Compute maximum "stress" appearing at any rod vertex in the structure
max([max(segmentStress) for segmentStress in l.rodStresses()[1]])

24.37145054902055

In [ ]:
# Output actuation torque
for forceScale in np.linspace(1.0, 0, 51):
    name = 'Pavilion14_{}'.format(forceScale)
    with suppress_stdout(): elastic_rods.compute_equilibrium(l, forceScale * externalForces, fixedVars=fixedVars)
    view.update(preserveExisting=True)
    writeRenderFiles(l, directory, name)
    writeActuators(l, np.where(torques > 0.1)[0], directory, name)

In [44]:
# Compute average "stress" appearing over all rod vertices in the structure
np.mean(sum(l.rodStresses()[1], []))

11.81760497978405

In [10]:
from open_linkage import open_linkage
def equilibriumSolver(tgtAngle, l, opts, fv):
    opts.beta = 1e-8
    opts.gradTol = 1e-4
    opts.useIdentityMetric = False
    return elastic_rods.compute_equilibrium(l, tgtAngle, options=opts, fixedVars=fv)
# open_linkage(l, driver, 2 * np.pi/3, 25, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=False);
open_linkage(l, driver, np.deg2rad(100) - l.averageJointAngle, 50, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=True);

target angle:  1.488765851951164
0	535371	715.795	715.795	1	0
1.2817892098632673	535363.9325515146	535363.9325515146	517889.9972146439	14374.611458677768	3099.3238781927694
target angle:  1.4940018397071468
0	540645	740.766	740.766	1	0
1.2870515460104046	540636.9582037174	540636.9582037174	522778.6493579041	14695.866296779297	3162.4425490341096
target angle:  1.4992378274631297
0	545963	771.904	771.904	1	0
1.292331576568166	545955.4714834855	545955.4714834853	527684.5758073273	15044.080128880072	3226.815547277965
target angle:  1.5044738152191126
0	551328	811.187	811.187	1	0
1.2976301396664038	551319.7434478868	551319.7434478868	532603.9536519897	15423.344693554447	3292.4451023425663
target angle:  1.5097098029750955
0	556739	861.239	861.239	1	0
1.3029482230546914	556730.0201063894	556730.0201063894	537532.1528801641	15838.539625998728	3359.3276002264656
target angle:  1.5149457907310784
0	562196	925.485	925.485	1	0
1.3082869952898577	562186.5163941816	562186.5163941816	542463.58224177